In [17]:
import torch
from copy import deepcopy

In [18]:
torch.manual_seed(0);

In [19]:
# from sparse_linear import *
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from src.model import *

In [20]:
def dense_to_sparse(dense_tensor: torch.Tensor) -> torch.sparse.FloatTensor:
    indices = dense_tensor.nonzero(as_tuple=True)
    values = dense_tensor[indices]
    indices = torch.stack(indices)

    sparse_tensor = torch.sparse.FloatTensor(indices, values, dense_tensor.size())
    return sparse_tensor

In [21]:
def convert_dense_to_sparse_network(model: nn.Module) -> nn.Module:
    """
    Converts a given dense neural network model to a sparse neural network model.

    This function recursively iterate through the given model and replaces all instances of 
    `nn.Linear` layers with `SparseLinear` layers

    Args:
        model (nn.Module): The dense neural network model to be converted.

    Returns:
        nn.Module: A new neural network model with sparse layers.
    """
    new_model = model.__class__()

    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            sparse_weight = dense_to_sparse(module.weight.data)
            sparse_bias = dense_to_sparse(module.bias.data)

            setattr(new_model, name, ExpandingLinear(sparse_weight, sparse_bias))
        else:
            setattr(new_model, name, convert_dense_to_sparse_network(module))
    return new_model

In [22]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = SimpleFCN()

In [23]:
sparse_model = convert_dense_to_sparse_network(model)

In [24]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = ExpandingLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

print(sparse_linear.weight_indices, "\n")
sparse_linear.replace(0, 6, 0)
sparse_linear.replace(0, 7, 0)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[0].weight_indeces, "\n")

sparse_linear.replace(0, 8, 1)
sparse_linear.replace(0, 9, 1)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[1].weight_indeces, "\n")

sparse_linear.replace(0, 11, 2)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[2].weight_indeces, "\n")



tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 8, 9]])
tensor([[0, 1],
        [6, 7]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[0, 1],
        [8, 9]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[ 0],
        [11]]) 



In [25]:
from nonlinearity_metrics import *

In [26]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [27]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [28]:
from edge_finder import *

In [29]:

ef = EdgeFinder(metrics[0], test_loader)
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges(sparse_linear, 4)
print("choose:", chosen_edges)

input torch.Size([32, 8])
embed torch.Size([2, 8])
tensor(indices=tensor([[0, 1],
                       [6, 7]]),
       values=tensor([0.3251, 0.3936]),
       size=(2, 8), nnz=2, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 10])
embed torch.Size([2, 10])
tensor(indices=tensor([[0, 1],
                       [8, 9]]),
       values=tensor([0.1743, 0.8579]),
       size=(2, 10), nnz=2, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 12])
embed torch.Size([1, 12])
tensor(indices=tensor([[ 0],
                       [11]]),
       values=tensor([0.5139]),
       size=(1, 12), nnz=1, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 8])
embed torch.Size([2, 8])
tensor(indices=tensor([[0, 1],
                       [6, 7]]),
       values=tensor([0.3251, 0.3936]),
       size=(2, 8), nnz=2, layout=torch.sparse_

In [30]:
child, parents = chosen_edges
for c, p in zip(child, parents):
    sparse_linear.replace(c, p, 3)

In [31]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 4,  5, 10, 11, 12, 13, 14, 15]])

In [32]:
ef = EdgeFinder(metrics[0], test_loader)
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges(sparse_linear, 4)
print("choose:", chosen_edges)

input torch.Size([32, 8])
embed torch.Size([2, 8])
tensor(indices=tensor([[0, 1],
                       [6, 7]]),
       values=tensor([0.3251, 0.3936]),
       size=(2, 8), nnz=2, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 10])
embed torch.Size([2, 10])
tensor(indices=tensor([[0, 1],
                       [8, 9]]),
       values=tensor([0.1743, 0.8579]),
       size=(2, 10), nnz=2, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 12])
embed torch.Size([1, 12])
tensor(indices=tensor([[ 0],
                       [11]]),
       values=tensor([0.5139]),
       size=(1, 12), nnz=1, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)
input torch.Size([32, 13])
embed torch.Size([4, 13])
tensor(indices=tensor([[0, 1, 2, 3],
                       [0, 3, 1, 2]]),
       values=tensor([0.9747, 0.0508, 0.8405, 0.2515]),
       size=(4, 1